In [ ]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori ,fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
import re

原始資料讀取和檢視

Input: user_review_Model_Name_with_category.csv

In [ ]:
modelname = 'MOBIUZ'

In [ ]:
raw = pd.read_csv('user_review_v1_'+modelname+'.csv') #根據model名修改檔名
raw = raw.drop(columns=['Unnamed: 0'])
#raw = raw.drop(columns='review_date_palce')
for i in range(len(raw)):
    raw['Asin'][i] = re.sub(r'[^a-zA-Z0-9]',"",raw['Asin'][i])
raw.index = range(len(raw.index))
raw

Merge Category with Asin

Input: ModelNameAsin_Category_list.csv

In [ ]:
Category_list = pd.read_csv(f'Asin_Category_{modelname}_list.csv')
Category_list = Category_list.drop(columns=['Unnamed: 0','Unnamed: 0.1'])
for i in range(len(Category_list)):
    for cl in ['Category','Sub_category']:
        Category_list[cl][i] = re.sub(r'[^a-zA-Z0-9]'," ",Category_list[cl][i])
    for cla in ['Asin']:
        Category_list[cla][i] = re.sub(r'[^a-zA-Z0-9]',"",Category_list[cla][i])

Category_list

In [ ]:
#print(Category_list['Asin'][0])
#print(raw['Asin'][0])
format = pd.merge(raw,Category_list)
clean_format = format.drop_duplicates()
clean_format.index = range(len(clean_format.index))
clean_format

In [ ]:
#原始Category數量
clean_format.value_counts('Category')

資料前處理: 1.根據商品名重新分類 2.時間格式處理

In [ ]:
re_cate_format = clean_format


In [ ]:
#調整時間格式
print('時間處理前:',type(re_cate_format['review_date_place'][0]),re_cate_format['review_date_place'][0])

for t in range(len(re_cate_format['review_date_place'])):
    time_raw = str(re_cate_format['review_date_place'][t]).split('·')[1]
    #若是中文時間格式則取消註解
    '''
    time_raw = time_raw.replace('年','-')
    time_raw = time_raw.replace('月','-')
    time_raw = time_raw.replace('日','')'''
    re_cate_format['review_date_place'][t] = time_raw
re_cate_format['review_date_place'] = pd.to_datetime(re_cate_format['review_date_place'])

print('時間處理後:',type(re_cate_format['review_date_place'][0]),re_cate_format['review_date_place'][0])


In [ ]:
re_cate_format.to_excel(modelname+'_Clean_Format.xlsx')

資料後處理 (時間篩選)

In [ ]:
start_time = '2018-01-01'
end_time = '2021-12-31'

In [ ]:
time_re_cate_format = re_cate_format.loc[(re_cate_format['review_date_place'] >= start_time) #時間起始點
                                       & (re_cate_format['review_date_place'] < end_time)] #時間終點
time_re_cate_format

In [ ]:
format_by_name = time_re_cate_format.groupby(['user_name'])
name_list = np.unique(time_re_cate_format['user_name'])
print('用戶名單:',name_list[:5],'...')
print('用戶數:',len(name_list))
#format_by_name.get_group(name_list[1])['Sub_category'][:3]

In [ ]:


male = 0
female =0
mt = 0
ft =0
for name in name_list:
    m=0
    f=0
    productname = format_by_name.get_group(name)['product_name']
    for pn in productname: #檢測購買男性商品與女性商品用戶數
        if 'Men\'s' in pn:
            m =1
            mt += 1
        elif 'Women\'s' in pn:
            f =1
            ft +=1  
    male += m
    female += f

print('直接購買男性商品的用戶數:',male,'總購買男性商品數',mt,'直接購買女性商品的用戶數:',female,'總購買女性商品',ft)

關聯式分析(User為購物籃)

Category Association Rule

In [ ]:
category_by_name =[]
for name in name_list:
    new_cate = format_by_name.get_group(name)['Category'] #個別用戶的Categorylist
    #print(name,new_cate)
    new_cate = list(new_cate)
    category_by_name.append(new_cate) #For Association Rule
te = TransactionEncoder()
te.ary = te.fit(category_by_name).transform(category_by_name)

In [ ]:
Association_df = pd.DataFrame(te.ary, columns=te.columns_)
frequent_items = fpgrowth(Association_df,min_support=0.1,use_colnames=True) #調整support值
frequent_items['length'] = frequent_items['itemsets'].apply(lambda x:len(x))
cate_frequent_format = pd.DataFrame(frequent_items[ (frequent_items['length'] == 1)])
#frequent_format = pd.DataFrame(frequent_items)
cate_frequent_format = cate_frequent_format.sort_values(by = 'support',ascending = False)
cate_frequent_format


In [ ]:
#出現Antecedents條件下出現consequents的機率為confidence
cate_asso_format = association_rules(frequent_items,metric='confidence',min_threshold=0.6) #調整confidence值
cate_asso_format = pd.DataFrame(cate_asso_format[ (cate_asso_format['lift'] > 1)]) #調整lift值
cate_asso_format

Sub_category Association rule

In [ ]:
sub_category_by_name =[]
for name in name_list:
    new_cate = format_by_name.get_group(name)['Sub_category'] #個別用戶的Categorylist
    #print(name,new_cate)
    new_cate = list(new_cate)
    sub_category_by_name.append(new_cate) #For Association Rule
te = TransactionEncoder()
te.ary = te.fit(sub_category_by_name).transform(sub_category_by_name)

In [ ]:
sub_Association_df = pd.DataFrame(te.ary, columns=te.columns_)
sub_frequent_items = fpgrowth(sub_Association_df,min_support=0.1,use_colnames=True) #調整support值
sub_frequent_items['length'] = sub_frequent_items['itemsets'].apply(lambda x:len(x))
sub_cate_frequent_format = pd.DataFrame(sub_frequent_items[ (sub_frequent_items['length'] == 1)])
#sub_frequent_format = pd.DataFrame(frequent_items)
sub_cate_frequent_format = sub_cate_frequent_format.sort_values(by = 'support',ascending = False)
sub_cate_frequent_format

In [ ]:
#出現Antecedents條件下出現consequents的機率為confidence
sub_cate_asso_format = association_rules(sub_frequent_items,metric='confidence',min_threshold=0.6) #調整confidence值
#sub_cate_asso_format = pd.DataFrame(sub_cate_asso_format[ (sub_cate_asso_format['lift'] > 1)]) #調整lift值
sub_cate_asso_format

In [ ]:
#將特別的itemset進行Breakdown
